In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../data/YAGO3-10-10/train.txt",delimiter="\t", header=None)
test = pd.read_csv("../data/YAGO3-10/test.txt",delimiter="\t", header=None)
validation = pd.read_csv("../data/YAGO3-10/valid.txt",delimiter="\t", header=None)
train.columns = ['Header', 'Tail', 'Relation']
test.columns = ['Header', 'Tail', 'Relation']
validation.columns = ['Header', 'Tail', 'Relation']

In [3]:
entities = pd.concat([train.Header, train.Tail,test.Header, test.Tail, validation.Header, validation.Tail])
entities = pd.DataFrame(entities,columns=['ID'])
entities = entities.groupby('ID').ID.agg(['count']).reset_index()


In [4]:
relations = pd.DataFrame([], columns=['ID'])
relations['ID'] = pd.concat([train.Relation,test.Relation,validation.Relation])
relations = relations.groupby('ID').ID.agg(['count']).reset_index()

In [5]:
test['Head_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Header[x]].tolist()[0]))
test['Tail_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Tail[x]].tolist()[0]))

train['Head_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Header[x]].tolist()[0]))
train['Tail_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Tail[x]].tolist()[0]))

validation['Head_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Header[x]].tolist()[0]))
validation['Tail_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Tail[x]].tolist()[0]))

train['Relation_idx'] = train.index.map(mapper=(lambda x : relations.index[relations.ID == train.Relation[x]].tolist()[0]))
test['Relation_idx'] = test.index.map(mapper=(lambda x : relations.index[relations.ID == test.Relation[x]].tolist()[0]))
validation['Relation_idx'] = validation.index.map(mapper=(lambda x : relations.index[relations.ID == validation.Relation[x]].tolist()[0]))

train = train.drop(['Header', 'Tail', 'Relation'], axis=1)
test = test.drop(['Header', 'Tail', 'Relation'], axis=1)
validation = validation.drop(['Header', 'Tail', 'Relation'], axis=1)

train = train.to_numpy()
test = test.to_numpy()
validation = validation.to_numpy()

In [6]:
totalWords = sum([freq[1] **(3/4) for freq in entities.values])     
entities['Probability'] = (entities['count']**(3/4) / totalWords)

totalWords = sum([freq[1] **(3/4) for freq in relations.values])     
relations['Probability'] = (relations['count']**(3/4) / totalWords)

In [7]:
entities.reset_index(level=0, inplace=True)
entities = entities.drop(['ID'], axis=1)
entities = entities.to_numpy()

relations.reset_index(level=0, inplace=True)
relations = relations.drop(['ID'], axis=1)
relations = relations.to_numpy()

In [9]:
np.save('../data/YAGO3-10_numpy/train.npy', train)
np.save('../data/YAGO3-10_numpy/validation.npy', validation)
np.save('../data/YAGO3-10_numpy/test.npy', test)

np.save('../data/YAGO3-10_numpy/entities.npy', entities)
np.save('../data/YAGO3-10_numpy/relations.npy', relations)